In [4]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.1 MB/s eta 0:00:00


In [ ]:
%%capture
%pip install requirements.txt

In [3]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [13]:
import os
import json
import glob
from string import Template
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep

#APIs
#import openai
from groq import Groq
from neo4j import GraphDatabase

#Langchain
from langchain_core.documents import Document
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
from langchain_groq import ChatGroq
#from langchain.llms import AzureOpenAI

#Streamlit
import streamlit as st
#from streamlit_chat import message

In [ ]:
# Load environment variables
load_dotenv()

In [ ]:
# OpenAI API configuration
#openai.api_type = "azure"
#openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.api_base = os.getenv("OPENAI_API_BASE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai_deployment = "chat-gpt35"

In [ ]:
# Neo4j configuration & constraints
graph_db = GraphDatabase.driver(os.getenv("NEO4J_CONNECTION_URL"), auth=(os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD")))

In [ ]:
# Groq API using LangChain and Gemma
llm_model = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"),
                     model_name = "llama3-8b-8192")

In [ ]:
# Define directories for different types of records
directories = {
    'patient_info': 'C:\Users\VISH NU\RAG_using_Graphs_and_Langchain\Data\patient_info',
    'patient_medical_history_briefs': 'C:\Users\VISH NU\RAG_using_Graphs_and_Langchain\Data\patient_info'
}

In [ ]:
def create_document_from_text(file_path, document_type):
    with open(file_path, 'r') as file:
        text_data = file.read().rstrip()
    return Document(page_content=text_data, metadata={'type': document_type})

def load_documents_from_directory(directory, document_type):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):  # Assuming the files are in .txt format
            file_path = os.path.join(directory, filename)
            document = create_document_from_text(file_path, document_type)
            documents.append(document)
    return documents

# Load documents from all directories
all_documents = []
for doc_type, dir_path in directories.items():
    documents = load_documents_from_directory(dir_path, doc_type)
    all_documents.extend(documents)

In [ ]:
# Function to call the Groq API
def process_gpt(file_prompt, system_msg):
    llm_model = Groq(groq_api_key=os.getenv("GROQ_API_KEY"))
    chat_completion = llm_model.chat.completions.create(
        max_tokens=15000,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_msg,
            }
            {
                "role": "user",
                "content": file_prompt,
            }
        ],
        model="llama3-8b-8192")
    results = chat_completion.choices[0].message.content
    sleep(8)
    return results

# Function to take a list of Document objects and a prompt template, and return a json-object of all the entities and relationships
def extract_entities_relationships(documents, prompt_template):
    start = timer()
    system_msg = "You are a helpful IT-project and account management expert who extracts information from documents."
    print(f"Running pipeline for {len(documents)} documents")
    results = []
    for i, document in enumerate(documents):
        print(f"Extracting entities and relationships for document {i+1}")
        try:
            text = document.page_content
            prompt = Template(prompt_template).substitute(ctext=text)
            result = process_gpt(prompt, system_msg=system_msg)
            results.append(json.loads(result))
        except Exception as e:
            print(f"Error processing document {i+1}: {e}")
    end = timer()
    print(f"Pipeline completed in {end-start} seconds")
    return results

# Function to take a json-object of entities and relationships and generate cypher query for creating those entities
def generate_cypher(json_obj):
    e_statements = []
    r_statements = []

    e_label_map = {}

    # loop through our json object
    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for document {i+1} of {len(json_obj)}")
        for entity in obj["entities"]:
            label = entity["label"]
            id = entity["id"]
            id = id.replace("-", "").replace("_", "")
            properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}

            cypher = f'MERGE (n:{label} {{id: "{id}"}})'
            if properties:
                props_str = ", ".join(
                    [f'n.{key} = "{val}"' for key, val in properties.items()]
                )
                cypher += f" ON CREATE SET {props_str}"
            e_statements.append(cypher)
            e_label_map[id] = label

        for rs in obj["relationships"]:
            src_id, rs_type, tgt_id = rs.split("|")
            src_id = src_id.replace("-", "").replace("_", "")
            tgt_id = tgt_id.replace("-", "").replace("_", "")

            src_label = e_label_map[src_id]
            tgt_label = e_label_map[tgt_id]

            cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
            r_statements.append(cypher)

    with open("cyphers.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements

# Final function to bring all the steps together
def ingestion_pipeline(all_documents, prompt_template):
    # Extracting the entities and relationships from each document, append into one json_object
    entities_relationships = extract_entities_relationships(all_documents, prompt_template)

    # Generate and execute cypher statements
    cypher_statements = generate_cypher(entities_relationships)
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            graph_db.execute_query(stmt)
        except Exception as e:
            with open("failed_statements.txt", "w") as f:
                f.write(f"{stmt} - Exception: {e}\n")

In [ ]:
## Using Langchain_experimental to create knowledge graphs by extracting entities and relationships within the document.
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer=LLMGraphTransformer(llm=llm)
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [ ]:
patient_info_prompt_template = """
From the Patient Info below, extract the following Entities & relationships described in the mentioned format
0. ALWAYS FINISH THE OUTPUT. Never send partial responses
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. Do not create new entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
    Entity Types:
    label:'Patient',id:string,name:string,age:int,gender:string //Patient information; `id` property is the unique identifier for the patient; 'name' is the patient's name; 'age' is the patient's age; 'gender' is the patient's gender
    label:'Contact',id:string,relation:string,name:string,phone:string,email:string //Contact information; `id` property is the unique identifier for the contact; 'relation' is the relationship to the patient; 'name' is the contact's name; 'phone' is the contact's phone number; 'email' is the contact's email address

2. Next generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective `id` property. Relationship property should be mentioned within brackets as comma-separated. They should follow these relationship types below. You will have to generate as many relationships as needed as defined below:
    Relationship types:
    patient|HAS_CONTACT|contact

3. The output should look like :
{
    "entities": [{"label":"Patient","id":string,"name":string,"age":int,"gender":string}],
    "relationships": ["patientid|HAS_CONTACT|contactid"]
}

Case Sheet:
$ctext
"""

patient_medical_history_prompt_template = """
From the Medical History below, extract the following Entities & relationships described in the mentioned format
0. ALWAYS FINISH THE OUTPUT. Never send partial responses
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. Do not create new entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
    Entity Types:
    label:'Patient',id:string,name:string //Patient information; `id` property is the unique identifier for the patient; 'name' is the patient's name
    label:'Condition',id:string,name:string,description:string //Medical Condition; `id` property is the unique identifier for the condition; 'name' is the name of the condition; 'description' is a brief description of the condition
    label:'Allergy',id:string,name:string //Allergy details; `id` property is the unique identifier for the allergy; 'name' is the name of the substance the patient is allergic to
    label:'Medication',id:string,name:string,dosage:string,frequency:string //Medication details; `id` property is the unique identifier for the medication; 'name' is the name of the medication; 'dosage' is the dosage of the medication; 'frequency' is the frequency of administration
    label:'LabTest',id:string,name:string,result:string,date:string //Lab Test details; `id` property is the unique identifier for the lab test; 'name' is the name of the lab test; 'result' is the result of the lab test; 'date' is the date the test was conducted
    label:'Visit',id:string,date:string,reason:string,doctor:string //Visit details; `id` property is the unique identifier for the visit; 'date' is the date of the visit; 'reason' is the reason for the visit; 'doctor' is the name of the doctor who attended the visit

2. Next generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective `id` property. Relationship property should be mentioned within brackets as comma-separated. They should follow these relationship types below. You will have to generate as many relationships as needed as defined below:
    Relationship types:
    patient|HAS_CONDITION|condition
    patient|HAS_ALLERGY|allergy
    patient|TAKES_MEDICATION|medication
    patient|UNDERGOES|labtest
    patient|HAS_VISIT|visit

3. The output should look like :
{
    "entities": [{"label":"Patient","id":string,"name":string}],
    "relationships": ["patientid|HAS_CONDITION|conditionid"],
    "relationships": ["patientid|HAS_ALLERGY|allergyid"],
    "relationships": ["patientid|TAKES_MEDICATION|medicationid"],
    "relationships": ["patientid|UNDERGOES|labtestid"],
    "relationships": ["patientid|HAS_VISIT|visitid"]
}

Case Sheet:
$ctext
"""


In [ ]:
folders = {
    'patient_info': patient_info_prompt_template,
    'patient_medical_history_briefs': patient_medical_history_prompt_template,
}

ingestion_pipeline(folders)

In [ ]:
cypher_generation_template = """
You are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, HAVING keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Patient, use `toLower(patient.name) contains 'john'`. To search for a Medical Condition, use 'toLower(condition.name) contains 'diabetes'`. To search for a Medication, use `toLower(medication.name) contains 'aspirin'`.)
5. Never use relationships that are not mentioned in the given schema
6. When asked about patients, match the properties using case-insensitive matching and the OR-operator, E.g., to find a patient named John Doe, use `toLower(patient.name) contains 'john' OR toLower(patient.name) contains 'doe'`.

schema: {
  Patient: {properties: ['id', 'name', 'age', 'gender']},
  Contact: {properties: ['id', 'relation', 'name', 'phone', 'email']},
  Condition: {properties: ['id', 'name', 'description']},
  Medication: {properties: ['id', 'name', 'dosage', 'frequency']},
  LabTest: {properties: ['id', 'name', 'result', 'date']},
  Visit: {properties: ['id', 'date', 'reason', 'doctor']},
  Relationships: [
    {Patient: 'HAS_CONTACT', Contact},
    {Patient: 'HAS_CONDITION', Condition},
    {Patient: 'TAKES_MEDICATION', Medication},
    {Patient: 'UNDERGOES', LabTest},
    {Patient: 'HAS_VISIT', Visit}
  ]
}

Examples:
Question: Which patients have undergone lab tests for diabetes?
Answer: ```MATCH (p:Patient)-[:UNDERGOES]->(lt:LabTest)
WHERE toLower(lt.name) contains 'diabetes' OR toLower(lt.result) contains 'diabetes'
RETURN p.name AS PatientName, lt.name AS LabTestName, lt.result AS Result```

Question: Which patients are taking aspirin?
Answer: ```MATCH (p:Patient)-[:TAKES_MEDICATION]->(m:Medication)
WHERE toLower(m.name) contains 'aspirin'
RETURN p.name AS PatientName, m.name AS MedicationName, m.dosage AS Dosage, m.frequency AS Frequency```

Question: {question}
"""

In [ ]:
# Cypher generation prompt
cypher_generation_template = """
You are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, HAVING keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Patient, use `toLower(patient.name) contains 'john'`. To search for a Medical Condition, use 'toLower(condition.name) contains 'diabetes'`. To search for a Medication, use `toLower(medication.name) contains 'aspirin'`.)
5. Never use relationships that are not mentioned in the given schema
6. When asked about patients, match the properties using case-insensitive matching and the OR-operator, E.g., to find a patient named John Doe, use `toLower(patient.name) contains 'john' OR toLower(patient.name) contains 'doe'`.

schema: {schema}

Examples:
Question: Which patients have undergone lab tests for diabetes?
Answer: ```MATCH (p:Patient)-[:UNDERGOES]->(lt:LabTest)
WHERE toLower(lt.name) contains 'diabetes' OR toLower(lt.result) contains 'diabetes'
RETURN p.name AS PatientName, lt.name AS LabTestName, lt.result AS Result```

Question: Which patients are taking aspirin?
Answer: ```MATCH (p:Patient)-[:TAKES_MEDICATION]->(m:Medication)
WHERE toLower(m.name) contains 'aspirin'
RETURN p.name AS PatientName, m.name AS MedicationName, m.dosage AS Dosage, m.frequency AS Frequency```

Question: {question}
"""


cypher_prompt = PromptTemplate(
    template = cypher_generation_template,
    input_variables = ["schema", "question"]
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.
Information:
{context}

Question: {question}
Helpful Answer:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

def query_graph(user_input):
    graph = Neo4jGraph(url=os.getenv("NEO4J_CONNECTION_URL"), username=os.getenv("NEO4J_USER"), password=os.getenv("NEO4J_PASSWORD"))
    chain = GraphCypherQAChain.from_llm(
        llm=llm_model,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True,
        cypher_prompt=cypher_prompt,
        qa_prompt=qa_prompt
        )
    result = chain(user_input)
    return result

In [ ]:
st.set_page_config(layout="wide")

if "user_msgs" not in st.session_state:
    st.session_state.user_msgs = []
if "system_msgs" not in st.session_state:
    st.session_state.system_msgs = []

title_col, empty_col, img_col = st.columns([2, 1, 2])

with title_col:
    st.title("Conversational Agent for Healthcare Clinics")
with img_col:
    st.image("https://dist.neo4j.com/wp-content/uploads/20210423062553/neo4j-social-share-21.png", width=200)

user_input = st.text_input("Enter your question", key="input")
if user_input:
    with st.spinner("Processing your question..."):
        st.session_state.user_msgs.append(user_input)
        start = timer()

        try:
            result = query_graph(user_input)

            intermediate_steps = result["intermediate_steps"]
            cypher_query = intermediate_steps[0]["query"]
            database_results = intermediate_steps[1]["context"]

            answer = result["result"]
            st.session_state.system_msgs.append(answer)
        except Exception as e:
            st.write("Failed to process question. Please try again.")
            print(e)

    st.write(f"Time taken: {timer() - start:.2f}s")

    col1, col2, col3 = st.columns([1, 1, 1])

    # Display the chat history
    with col1:
        if st.session_state["system_msgs"]:
            for i in range(len(st.session_state["system_msgs"]) - 1, -1, -1):
                message(st.session_state["system_msgs"][i], key = str(i) + "_assistant")
                message(st.session_state["user_msgs"][i], is_user=True, key=str(i) + "_user")

    with col2:
        if cypher_query:
            st.text_area("Last Cypher Query", cypher_query, key="_cypher", height=240)

    with col3:
        if database_results:
            st.text_area("Last Database Results", database_results, key="_database", height=240)
